In [2]:
ls

drive/  sample_data/


In [3]:
cd drive

/content/drive


In [ ]:
ls

'My Drive'/


In [4]:
cd My\ Drive

/content/drive/My Drive


In [5]:
cd Colab

/content/drive/My Drive/Colab


In [ ]:
ls

image-classification-keras/


In [6]:
cd image-classification-keras/

/content/drive/My Drive/Colab/image-classification-keras


In [7]:
ls

data/  examples/  graph.png  Mask_1209.model  Test.ipynb  Train.ipynb


In [8]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

In [9]:
# initialize the number of epochs to train for, initia learning rate,
# and batch size
EPOCHS = 50
INIT_LR = 0.0001
Batch_Size = 32

In [10]:
#https://github.com/prajnasb/observations/tree/master/experiements/data :: Dataset
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images("data")))
random.seed(42)
random.shuffle(imagePaths)

print (imagePaths)

[INFO] loading images...
['data/without_mask/154.jpg', 'data/without_mask/augmented_image_200.jpg', 'data/without_mask/440.jpg', 'data/without_mask/77.jpg', 'data/without_mask/450.jpg', 'data/without_mask/43.jpg', 'data/without_mask/423.jpg', 'data/with_mask/227-with-mask.jpg', 'data/with_mask/augmented_image_247.jpg', 'data/without_mask/augmented_image_96.jpg', 'data/with_mask/106-with-mask.jpg', 'data/with_mask/augmented_image_98.jpg', 'data/without_mask/415.jpg', 'data/with_mask/416-with-mask.jpg', 'data/with_mask/348-with-mask.jpg', 'data/with_mask/augmented_image_174.jpg', 'data/with_mask/augmented_image_82.jpg', 'data/with_mask/417-with-mask.jpg', 'data/without_mask/248.jpg', 'data/without_mask/388.jpg', 'data/with_mask/112-with-mask.jpg', 'data/without_mask/451.jpg', 'data/with_mask/187-with-mask.jpg', 'data/without_mask/augmented_image_317.jpg', 'data/without_mask/431.jpg', 'data/with_mask/augmented_image_147.jpg', 'data/without_mask/297.jpg', 'data/with_mask/144-with-mask.jpg'

In [11]:
# loop over the input images
for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	print (imagePath)
	image = cv2.resize(image, (28, 28))
	image = img_to_array(image)
	data.append(image)
	#print(data)
	
  # extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[1]
	print(label+"\n")
	label = 1 if label == "with_mask" else 0
	labels.append(label)

data/without_mask/154.jpg
without_mask

data/without_mask/augmented_image_200.jpg
without_mask

data/without_mask/440.jpg
without_mask

data/without_mask/77.jpg
without_mask

data/without_mask/450.jpg
without_mask

data/without_mask/43.jpg
without_mask

data/without_mask/423.jpg
without_mask

data/with_mask/227-with-mask.jpg
with_mask

data/with_mask/augmented_image_247.jpg
with_mask

data/without_mask/augmented_image_96.jpg
without_mask

data/with_mask/106-with-mask.jpg
with_mask

data/with_mask/augmented_image_98.jpg
with_mask

data/without_mask/415.jpg
without_mask

data/with_mask/416-with-mask.jpg
with_mask

data/with_mask/348-with-mask.jpg
with_mask

data/with_mask/augmented_image_174.jpg
with_mask

data/with_mask/augmented_image_82.jpg
with_mask

data/with_mask/417-with-mask.jpg
with_mask

data/without_mask/248.jpg
without_mask

data/without_mask/388.jpg
without_mask

data/with_mask/112-with-mask.jpg
with_mask

data/without_mask/451.jpg
without_mask

data/with_mask/187-with-mask.

In [12]:
data = np.array(data)
labels = np.array(labels)
#np.set_printoptions(threshold=np.inf)
print (data[0])
#print(data)
print(data.shape)
print(labels.size)
print(labels)
print(label)

[[[ 70. 100.  88.]
  [ 33.  60.  50.]
  [ 48.  71.  66.]
  ...
  [ 51.  85.  72.]
  [ 75. 123. 113.]
  [ 49.  97.  70.]]

 [[ 61. 102.  82.]
  [ 29.  72.  58.]
  [ 48.  81.  74.]
  ...
  [ 32.  64.  63.]
  [ 46.  85.  87.]
  [ 53. 109. 103.]]

 [[ 84. 119. 108.]
  [ 43.  59.  57.]
  [ 70.  89.  88.]
  ...
  [ 55.  67.  65.]
  [ 42.  56.  57.]
  [ 55.  85.  85.]]

 ...

 [[160. 157. 135.]
  [ 93. 100.  79.]
  [123. 132. 112.]
  ...
  [102.  95.  74.]
  [ 88.  65.  50.]
  [115.  89.  75.]]

 [[ 94. 105.  79.]
  [101. 109.  88.]
  [ 71.  80.  60.]
  ...
  [100. 111.  83.]
  [104. 112.  89.]
  [125. 123. 102.]]

 [[ 87.  98.  74.]
  [109. 119.  96.]
  [ 53.  62.  42.]
  ...
  [ 88. 103.  75.]
  [ 98. 109.  86.]
  [105. 113.  90.]]]
(1354, 28, 28, 3)
1354
[0 0 0 ... 1 1 0]
0


In [13]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
print(data[0])

[[[0.2745098  0.39215686 0.34509804]
  [0.12941176 0.23529412 0.19607843]
  [0.18823529 0.27843137 0.25882353]
  ...
  [0.2        0.33333333 0.28235294]
  [0.29411765 0.48235294 0.44313725]
  [0.19215686 0.38039216 0.2745098 ]]

 [[0.23921569 0.4        0.32156863]
  [0.11372549 0.28235294 0.22745098]
  [0.18823529 0.31764706 0.29019608]
  ...
  [0.1254902  0.25098039 0.24705882]
  [0.18039216 0.33333333 0.34117647]
  [0.20784314 0.42745098 0.40392157]]

 [[0.32941176 0.46666667 0.42352941]
  [0.16862745 0.23137255 0.22352941]
  [0.2745098  0.34901961 0.34509804]
  ...
  [0.21568627 0.2627451  0.25490196]
  [0.16470588 0.21960784 0.22352941]
  [0.21568627 0.33333333 0.33333333]]

 ...

 [[0.62745098 0.61568627 0.52941176]
  [0.36470588 0.39215686 0.30980392]
  [0.48235294 0.51764706 0.43921569]
  ...
  [0.4        0.37254902 0.29019608]
  [0.34509804 0.25490196 0.19607843]
  [0.45098039 0.34901961 0.29411765]]

 [[0.36862745 0.41176471 0.30980392]
  [0.39607843 0.42745098 0.34509804]


In [14]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25)

In [15]:
print(testY)
print(testY.shape)

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1
 1 1 0 1 0 1 0 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 0 0 1 0 1 1 1 0 1 0 0 1 1 0 0
 1 1 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 0 1 1 1 0
 0 0 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 1 1 0 0 0 1 1 1 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0
 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 0 1 1 1 1 0 0 0 1
 1 0 1 0 0 1 1 1 0 0 0 0 0 1 0 1 1 1 0 1 0 1 0 1 0 0 0 1 0 1 0 0 1 0 1 1 0
 0 0 0 1 0 0 0 1 1 0 0 0 1 1 1 1 1 1 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 1 1
 0 0 1 0 0 0 0 1 1 1 1 0 1 1 0 1 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 0 0 1 1
 0 1 0 0 0 1]
(339,)


In [16]:
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [17]:
print(testY)
print(testY.shape)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 

In [33]:
print("[INFO] compiling model...")

model = Sequential()
inputShape = (28, 28, 3)

# first set of CONV => RELU => POOL layers
model.add(Conv2D(20, (5, 5), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# second set of CONV => RELU => POOL layers
model.add(Conv2D(50, (5, 5), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# first (and only) set of FC => RELU layers
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))

# softmax classifier
model.add(Dense(2))
model.add(Activation("softmax"))

opt = Adam(lr=INIT_LR) 
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

print(model.summary())

[INFO] compiling model...
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 28, 28, 20)        1520      
_________________________________________________________________
activation_21 (Activation)   (None, 28, 28, 20)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 14, 14, 50)        25050     
_________________________________________________________________
activation_22 (Activation)   (None, 14, 14, 50)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 7, 7, 50)          0         
_________________________________________________________________
flatten_5 (Flatten)         

In [34]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [35]:
# train the network
print("[INFO] training network...")
H = model.fit(x=aug.flow(trainX, trainY, batch_size=Batch_Size), validation_data=(testX, testY), epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/50
32/32 [==============================] - 1s 28ms/step - loss: 0.6413 - accuracy: 0.6138 - val_loss: 0.5265 - val_accuracy: 0.8230
Epoch 2/50
32/32 [==============================] - 1s 24ms/step - loss: 0.4742 - accuracy: 0.8197 - val_loss: 0.3717 - val_accuracy: 0.8673
Epoch 3/50
32/32 [==============================] - 1s 25ms/step - loss: 0.3870 - accuracy: 0.8433 - val_loss: 0.2723 - val_accuracy: 0.9027
Epoch 4/50
32/32 [==============================] - 1s 24ms/step - loss: 0.2971 - accuracy: 0.8828 - val_loss: 0.2503 - val_accuracy: 0.9233
Epoch 5/50
32/32 [==============================] - 1s 23ms/step - loss: 0.2545 - accuracy: 0.9212 - val_loss: 0.2163 - val_accuracy: 0.9145
Epoch 6/50
32/32 [==============================] - 1s 24ms/step - loss: 0.2213 - accuracy: 0.9182 - val_loss: 0.1732 - val_accuracy: 0.9381
Epoch 7/50
32/32 [==============================] - 1s 24ms/step - loss: 0.2124 - accuracy: 0.9192 - val_loss: 0.1504 - val_acc

In [ ]:
# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, batch_size=Batch_Size, validation_data=(testX, testY), epochs=EPOCHS, verbose=1)

[INFO] training network...
Epoch 1/50
32/32 [==============================] - 0s 6ms/step - loss: 2.6297e-05 - accuracy: 1.0000 - val_loss: 0.0503 - val_accuracy: 0.9853
Epoch 2/50
32/32 [==============================] - 0s 6ms/step - loss: 2.1898e-05 - accuracy: 1.0000 - val_loss: 0.0553 - val_accuracy: 0.9882
Epoch 3/50
32/32 [==============================] - 0s 5ms/step - loss: 2.5489e-05 - accuracy: 1.0000 - val_loss: 0.0589 - val_accuracy: 0.9853
Epoch 4/50
32/32 [==============================] - 0s 5ms/step - loss: 2.1847e-05 - accuracy: 1.0000 - val_loss: 0.0478 - val_accuracy: 0.9853
Epoch 5/50
32/32 [==============================] - 0s 5ms/step - loss: 2.2286e-05 - accuracy: 1.0000 - val_loss: 0.0501 - val_accuracy: 0.9853
Epoch 6/50
32/32 [==============================] - 0s 5ms/step - loss: 2.1556e-05 - accuracy: 1.0000 - val_loss: 0.0503 - val_accuracy: 0.9853
Epoch 7/50
32/32 [==============================] - 0s 5ms/step - loss: 1.9904e-05 - accuracy: 1.0000 - val_l

In [ ]:
# save the model to disk
print("[INFO] serializing network...")
model.save("Mask_1209.model", save_format="h5")

[INFO] serializing network...


In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training/Testing Loss and Accuracy on Mask/Unmask")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("graph")